## HOLO version of TOML config file workbook

In [ ]:
import pyopia.background
import pyopia.classify
import pyopia.instrument.silcam
import pyopia.instrument.holo
import pyopia.io
import pyopia.pipeline
import pyopia.plotting
import pyopia.process
import pyopia.statistics
import exampledata

import xarray
import matplotlib.pyplot as plt

## Load a TOML config file containing all setttings, and pipeline steps

This creates a dict of all settings, which can be modified or writte in the same way as any other dictionary in python. This has a slightly different structure that the old 'steps' dict, but follows the same principle (look in the example config.toml file used below for more details).

In [ ]:
toml_settings = pyopia.io.load_toml('config-holo.toml')
toml_settings

## Run the pipeline

(same as before, but now giving Pipeline() the toml_setting dict)

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(toml_settings)

# Load an image (from the test suite)
filename = 'holo_test_data_01/001-2043.pgm'

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

## Load the data again

Then later, we can load the data again from NetCDF using xarray

xarray DataSets are presented nicely in notebooks (rendered poorly on GitHub's webinterface)

In [ ]:
with xarray.open_dataset('proc/test-STATS.nc') as xstats:
    xstats.load()

xstats

## Alter settings and re-process

What if we wanted to re-process this dataset with a different segmentation threshold?

### Get the TOML steps from the xarray DataSet

In [ ]:
toml_steps = pyopia.pipeline.steps_from_xstats(xstats)
toml_steps

### Alter the setting we want to change

In [ ]:
toml_steps['steps']['focus']['stacksummary_function'] = 'std_map'

### re-process the pipeline

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(toml_steps)

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

### Further analysis

At this point we could write this to disc again (using the pyopia.io.write_stats function)

and/or we could build a new, correctly formatted, xarray for immediate use (which we will do here):

In [ ]:
xstats_modified = pyopia.io.make_xstats(stats, toml_steps)
xstats_modified

### Plotting

We can plot directly from xarray in exactly the same way as from the Pandas DataFrame (so it doesn't matter which you use here). The benefit of 'xstats' as an xarray is that it now contains it's own metadata

In [ ]:
dias, vd = pyopia.statistics.vd_from_stats(xstats, pyopia.pipeline.steps_from_xstats(xstats)['general']['pixel_size'])

plt.plot(dias, vd, label=f"Threshold={pyopia.pipeline.steps_from_xstats(xstats)['steps']['focus']['threshold']}")
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

dias_modified, vd_modified = pyopia.statistics.vd_from_stats(xstats_modified, pyopia.pipeline.steps_from_xstats(xstats_modified)['general']['pixel_size'])

plt.plot(dias_modified, vd_modified, '--', label=f"Threshold={pyopia.pipeline.steps_from_xstats(xstats_modified)['steps']['focus']['threshold']}")

plt.legend()

# Remaining things to do/fix that are not covered in this notebook:

* create a pyopia.main that does administration (instead of this notebook) and has command-line inputs with config.toml file as input
* a pyopia.main should automatically setup the pipeline and process the images in settings['general']['raw_files'] file list in pipeline.run()